In [ ]:
# LIBRARIES
import numpy as np
import cv2
import os
from PIL import Image

In [ ]:
# --- path of the folder containing the images
sourcePath = 'd://z/research/03-roi/20230119/ARABIKA'
# --- path of the folder that will contain the modified image
outputPath = 'd://z/research/04-segmented/20230119/ARABIKA'

In [ ]:
def dip_bgremoval(sourcePath, outputPath):
    for imagePath in os.listdir(sourcePath):

        # --- imagePath contains name of the image
        inputPath = os.path.join(sourcePath, imagePath)

        # --- inputPath contains the full directory name
        #mg = Image.open(inputPath)
        #filename = 'c://cv/research/03-roi/20230110/IMG-20230110_A_20230110_161122.png'
        img = cv2.imread(inputPath)
        h,w,c = img.shape
        image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        #rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        rectangle = (0, 0, h-1, w-1)
        mask = np.zeros(img.shape[:2], np.uint8)
        bgdModel = np.zeros((1, 65), np.float64)
        fgdModel = np.zeros((1, 65), np.float64)
        cv2.grabCut(img, mask, rectangle, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)
        mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
        image_ = img * mask2[:, :, np.newaxis]

        tmp = cv2.cvtColor(image_, cv2.COLOR_BGR2GRAY)
        thotsu, imgotsu = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        b, g, r, = cv2.split(image_)
        rgba = [r, g, b, imgotsu]
        nobg = cv2.merge(rgba, 4)
        image2 = cv2.cvtColor(nobg, cv2.COLOR_RGB2BGR)

        print("Obtained threshold: ", thotsu)
        #cv2.imshow('Background-Removed Image', image2)

        segmented_filename = os.path.splitext(inputPath)[0] + '_segmented.png'
        segmented_basename = os.path.basename(segmented_filename)
        cv2.imwrite(os.path.join(outputPath, segmented_basename), image2)

        #cv2.waitKey(0)
        #cv2.destroyAllWindows

# Ref : https://learnopencv.com/otsu-thresholding-with-opencv/

In [ ]:
if __name__ == '__main__':
    dip_bgremoval(sourcePath, outputPath)